In [6]:
from tensorflow import keras, expand_dims
import numpy as np
model_name = 'VisFin_v6.h5'

In [7]:
def make_new_model(dropout = 0):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(9, kernel_size = (9,18),padding='same', input_shape=(201, 43, 1), activation='elu'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Conv2D(9, kernel_size = (9,18), padding='same', activation='elu'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.MaxPooling2D())
    model.add(keras.layers.Conv2D(9, kernel_size = (6,12),padding='same', activation='elu'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.MaxPooling2D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(84, activation='elu'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(42, activation='elu'))
    model.add(keras.layers.Dense(7,activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [8]:
from modules.data_feed import *
modeling = PrepareData('daily__us__nasdaq_stocks.csv', '<DATE>', skip=5)
modeling.clean_db()

In [9]:
try:
    VisCV = keras.models.load_model(model_name)
    print('Loading!')
except OSError:
    VisCV = make_new_model(dropout=0.3)
    print('Compiling!')

Compiling!


In [10]:
# TRAINING
data = modeling.mass_hacking(tick=1, horizon=42)
tr_data = expand_dims(np.array(data[0]),axis=-1)
tr_key = np.array(data[1])
if tr_data.shape[0] > 0:
    VisCV.fit(tr_data, tr_key, epochs=3, verbose=2)
VisCV.save(model_name)


Epoch 1/3
70/70 - 3s - loss: 1.3191 - accuracy: 0.5590
Epoch 2/3
70/70 - 2s - loss: 0.9890 - accuracy: 0.6435
Epoch 3/3
70/70 - 2s - loss: 0.7730 - accuracy: 0.7036


In [11]:
# EVALUATE
data = modeling.mass_hacking(tick=0, horizon=42)
tr_data = expand_dims(np.array(data[0]),axis=-1)
tr_key = np.array(data[1])
eval_ = VisCV.evaluate(tr_data,tr_key, verbose=1)
print(model_name+': ',eval_[0], eval_[1])

76/76 [==============================] - 1s 13ms/step - loss: 1.5452 - accuracy: 0.4561
VisFin_v6.h5:  1.5451676845550537 0.45608946681022644


In [12]:
# # # convert to JS
# import tensorflowjs
# # tensorflowjs.converters.save_keras_model(VisFin0, 'js_format\\VisFin_0')
# # tensorflowjs.converters.save_keras_model(VisFin1, 'js_format\\VisFin_1')
# # tensorflowjs.converters.save_keras_model(VisFin2, 'js_format\\VisFin_2')
# # tensorflowjs.converters.save_keras_model(VisFin3, 'js_format\\VisFin_3')
